In [1]:
%load_ext autoreload
%autoreload 2

import math
import random
import tempfile
import tensorflow as tf
import time

import numpy as np

from collections import defaultdict

from tf_rl.controller import KerasDDPG
from tf_rl.models     import KERASMLP
from tf_rl.simulation import SinglePendulum
from tf_rl            import simulate

from keras import backend as K

SINGLE_PENDULUM_PARAMS = {
    'g_ms2': 9.8, # acceleration due to gravity, in m/s^2
    'l1_m': 1.0, # length of pendulum 1 in m
    'm1_kg': 1.0, # mass of pendulum 1 in kg
    'damping': 0.2,
    'max_control_input': 1.5
}

Using TensorFlow backend.


In [2]:
actor = KERASMLP(SinglePendulum.observation_size, [512, 256, 1], ['relu', 'relu', 'tanh'])
critic = KERASMLP(SinglePendulum.observation_size+SinglePendulum.action_size,[512, 256, 1],['relu', 'relu', 'linear'])

In [3]:
current_controller = KerasDDPG(SinglePendulum.observation_size, 
                               SinglePendulum.action_size, actor, critic, discount_rate=0.99, 
                               exploration_period=5000)

In [6]:
fast_mode = False

if fast_mode:
    FPS, SPEED, RES = 5, 20.0, 0.03
else:
    FPS, SPEED, RES = 30, 1., 0.001

try:
    while True:
        d = SinglePendulum(SINGLE_PENDULUM_PARAMS)
        simulate(d, current_controller, fps=FPS,
                 simulation_resolution=RES,
                 action_every=10,
                 disable_training=False,
                 visualize=True)
except KeyboardInterrupt:
    print("Interrupted")

<tf_rl.utils.svg.Scene instance at 0x7f9bef9c8638>

Starting training step 16801 at 2016-02-26 23:23:24
Interrupted


In [ ]:
c_grad = [K.gradients(K.sum(critic.model.get_output(train=False)), critic.model.get_input(train=False))[0]]

In [ ]:
s1,s2,s3,s4,act_grad = tf.split(1,5,c_grad[0])

In [ ]:
minibatch_size=32

In [ ]:
minibatch = K.variable(minibatch_size)

In [ ]:
actor.model.trainable_weights

In [ ]:
a_grad_1 = [(K.gradients(K.sum(actor.model.get_output(train=False),axis=1), z)[0] for z in actor.model.trainable_weights)]

In [ ]:
a_grad_1_2 = [a_grad_1 / minibatch]

In [ ]:
simple_test = [K.sum(actor.model.get_output(train=False),axis=1)]

In [ ]:
simp_test_func = K.function(inputs=[actor.model.get_input(train=False)], outputs=simple_test)

In [ ]:
simp_test_func([test_input_a])

In [ ]:
a_grad_2 = [K.gradients(K.sum(actor.model.get_output(train=False)), actor.model.trainable_weights)[0]]

In [ ]:
test_update_1 = K.function(inputs=[critic.model.get_input(train=False),actor.model.get_input(train=False)], outputs=a_grad_1)

In [ ]:
test_update_2 = K.function(inputs=[critic.model.get_input(train=False),actor.model.get_input(train=False)], outputs=a_grad_2)

In [ ]:
test_input_c = np.random.rand(32,5)

In [ ]:
test_input_a = np.random.rand(32,4)

In [ ]:
p_grad_1 = test_update_1([test_input_c, test_input_a])

In [ ]:
p_grad_2 = test_update_2([test_input_c, test_input_a])

In [ ]:
p_grad_1

In [ ]:
p_grad_2

In [ ]:
current_controller.plot_critic_value_function()

In [ ]:
current_controller.plot_actor_policy()

In [ ]:
current_controller.plot_critic_value_function()

In [ ]:
current_controller.plot_actor_policy()

In [ ]:
current_controller.bellman_error[-10:]

In [ ]:
print len(current_controller.experience)

In [ ]:
current_controller.restore_checkpoint('/home/mderry/tensorflow-deepq/notebooks/logs/pendulum_checkpoint_6001')

In [5]:
fast_mode = False

if fast_mode:
    FPS, SPEED, RES = 5, 20.0, 0.03
else:
    FPS, SPEED, RES = 30, 1., 0.001

SINGLE_PENDULUM_PARAMS = {
    'g_ms2': 9.8, # acceleration due to gravity, in m/s^2
    'l1_m': 1.0, # length of pendulum 1 in m
    'm1_kg': 1.0, # mass of pendulum 1 in kg
    'damping': 0.2,
    'max_control_input': 1.5
}
d = SinglePendulum(SINGLE_PENDULUM_PARAMS)
try:
    while True:
        simulate(d, current_controller, fps=FPS, simulation_resolution=RES, wait=False, action_every=10, disable_training=True, ignore_exploration=True)
except KeyboardInterrupt:
    print("Interrupted")

<tf_rl.utils.svg.Scene instance at 0x7faa61462758>

Interrupted


In [ ]:
from pympler import tracker

In [ ]:
mem_tracker = tracker.SummaryTracker()

In [ ]:
mem_tracker.print_diff()